# Asistente Legal (LangChain + LLM)

Este notebook implementa el proyecto: "Asistente legal / normativo" para consultar leyes colombianas.

Objetivo de esta sección: preparar el entorno e importar las librerías necesarias. Sigue las celdas en orden.


In [30]:
##!uv add "langchain[openai]" dotenv ipywidgets chromadb faiss-cpu PyPDF2

## installar dependencias con pip
#%pip install -q "langchain[openai]" python-dotenv ipywidgets chromadb faiss-cpu PyPDF2
#!pip install groq langchain-groq


In [1]:
# Imports y configuración inicial
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Import the LLM provider module
from llm_providers import get_llm_client, LLMProviderFactory

# Get LLM provider from environment (default: groq)
provider = os.getenv('LLM_PROVIDER', 'groq').lower()
print(f"Available providers: {LLMProviderFactory.get_available_providers()}")
print(f"Using provider: {provider}")

# Initialize LLM client based on provider
llm = get_llm_client(provider)
print(f"LLM client initialized successfully!")

# Prompt template de ejemplo
prompt = PromptTemplate(
    input_variables=["question"],
    template="Eres un asistente legal colombiano. Responde: {question}"
)


Available providers: ['openai', 'openrouter', 'ollama', 'groq']
Using provider: openrouter
LLM client initialized successfully!
LLM client initialized successfully!


In [2]:
# Loaders: widget FileUpload y helpers para parsear archivos subidos y leer `data/`
import ipywidgets as widgets
from IPython.display import display
import os
from io import BytesIO

# Widget para subir archivos
uploader = widgets.FileUpload(accept='.txt,.md,.pdf', multiple=True)
display(widgets.VBox([widgets.Label("Sube archivos legales (.pdf, .txt, .md)"), uploader]))

# Output para previsualización
output = widgets.Output()
display(output)


def parse_uploaded_files(uploader_widget):
    """Devuelve una lista de dicts {'filename', 'text'} extraídos de los archivos subidos."""
    docs = []
    # La estructura de uploader.value depende del frontend; manejar ambos casos
    items = getattr(uploader_widget, 'value', {}) or {}
    # En algunos entornos items es lista, en otros dict
    try:
        iterator = items.items()
    except Exception:
        # intentamos tratar como lista
        iterator = [(f.get('name', f"file_{i}"), f) for i, f in enumerate(items)]

    for name, fileinfo in iterator:
        # fileinfo puede ser dict con 'content' o directamente bytes
        content = fileinfo.get('content') if isinstance(fileinfo, dict) else fileinfo
        text = ''
        if name.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(BytesIO(content))
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] No se pudo leer PDF ({name}): {e}"
        else:
            try:
                if isinstance(content, bytes):
                    text = content.decode('utf-8')
                else:
                    text = str(content)
            except Exception as e:
                text = f"[ERROR] decodificando {name}: {e}"
        docs.append({'filename': name, 'text': text})
    return docs


def on_preview_clicked(b):
    with output:
        output.clear_output()
        docs = parse_uploaded_files(uploader)
        if not docs:
            print('No se han encontrado archivos subidos.')
            return
        for d in docs:
            print(f"--- {d['filename']} ---")
            print(d['text'][:1500])
            print('\n')

preview_btn = widgets.Button(description='Previsualizar archivos')
preview_btn.on_click(on_preview_clicked)
display(preview_btn)


def load_local_docs(data_dir='data'):
    """Lee archivos de `data/` y devuelve lista de dicts {'filename','text'}"""
    docs = []
    if not os.path.exists(data_dir):
        print(f"No existe la carpeta '{data_dir}'. Crea la carpeta y añade documentos de ejemplo.")
        return docs
    for fn in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, fn)
        if not os.path.isfile(path):
            continue
        text = ''
        if fn.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(path)
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] leyendo PDF {fn}: {e}"
        else:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except Exception as e:
                text = f"[ERROR] leyendo {fn}: {e}"
        docs.append({'filename': fn, 'text': text})
    return docs

# Ejemplo de uso: docs = load_local_docs('data')
# print(docs[0]['text'][:500]) if docs else print('No hay documentos en data/')


Output()

Button(description='Previsualizar archivos', style=ButtonStyle())

## Sección: Loaders (carga de documentos)

En esta sección implementaremos los loaders que permiten: (1) subir archivos locales mediante un widget, y (2) cargar archivos desde la carpeta `data/` del repositorio. Más adelante convertiremos estos textos en chunks y los indexaremos en una vector DB.

Instrucciones rápidas:
- Usa el widget para subir `.pdf`, `.txt` o `.md`.
- Pulsa "Previsualizar archivos" para ver el texto extraído (o errores si falta una dependencia de PDF).

In [28]:
# Preprocesamiento: splitters y creación de chunks
# Carga documentos desde `data/`, aplica un splitter y guarda los chunks en `DOCUMENT_CHUNKS`.
from collections import Counter

# Intentar importar el splitter de LangChain; si la API cambia, manejar el fallback
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    try:
        # versión alternativa
        from langchain.text_splitter import CharacterTextSplitter as RecursiveCharacterTextSplitter
    except Exception:
        RecursiveCharacterTextSplitter = None


def create_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """Devuelve lista de dicts {'source','chunk_index','text'}"""
    if RecursiveCharacterTextSplitter is None:
        raise ImportError("No se encontró RecursiveCharacterTextSplitter. Instala langchain actualizado.")
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []
    for d in docs:
        text = d.get('text') or ''
        if not text.strip():
            # texto vacío o error al leer
            continue
        texts = splitter.split_text(text)
        for i, t in enumerate(texts):
            all_chunks.append({'source': d.get('filename', 'unknown'), 'chunk_index': i, 'text': t})
    return all_chunks

# Cargar documentos locales desde data/
DOCUMENTS = load_local_docs('data')
print(f"Documentos cargados: {len(DOCUMENTS)}")

# Crear chunks
DOCUMENT_CHUNKS = create_chunks(DOCUMENTS, chunk_size=1000, chunk_overlap=200) if DOCUMENTS else []
print(f"Total chunks generados: {len(DOCUMENT_CHUNKS)}")

# Mostrar conteo por documento y un ejemplo
if DOCUMENT_CHUNKS:
    per_doc = Counter([c['source'] for c in DOCUMENT_CHUNKS])
    for doc, cnt in per_doc.items():
        print(f" - {doc}: {cnt} chunks")
    print('\nEjemplo (primer chunk, 500 chars):')
    print(DOCUMENT_CHUNKS[0]['text'][:500])
else:
    print('No hay chunks (documentos vacíos o problema al leer los archivos).')

# Ahora `DOCUMENTS` y `DOCUMENT_CHUNKS` están listos para embeddings / indexado.


Object ID 7606,0 ref repaired
Object ID 1,0 ref repaired
Object ID 1,0 ref repaired
Object ID 2,0 ref repaired
Object ID 2,0 ref repaired
Object ID 7582,0 ref repaired
Object ID 7582,0 ref repaired
Object ID 7584,0 ref repaired
Object ID 7584,0 ref repaired
Object ID 7583,0 ref repaired
Object ID 7583,0 ref repaired
Object ID 7589,0 ref repaired
Object ID 7589,0 ref repaired
Object ID 7591,0 ref repaired
Object ID 7591,0 ref repaired
Object ID 7590,0 ref repaired
Object ID 7590,0 ref repaired
Object ID 7596,0 ref repaired
Object ID 7596,0 ref repaired
Object ID 7598,0 ref repaired
Object ID 7598,0 ref repaired
Object ID 7597,0 ref repaired
Object ID 7597,0 ref repaired
Object ID 4,0 ref repaired
Object ID 4,0 ref repaired
Object ID 7604,0 ref repaired
Object ID 7604,0 ref repaired
Object ID 6,0 ref repaired
Object ID 6,0 ref repaired
Object ID 8,0 ref repaired
Object ID 8,0 ref repaired
Object ID 10,0 ref repaired
Object ID 10,0 ref repaired
Object ID 12,0 ref repaired
Object ID 12,0 r

Documentos cargados: 2
Total chunks generados: 1215
 - Constitución_Política_1_de_1991_Asamblea_Nacional_Constituyente.pdf: 1050 chunks
 - reglamento_academi_pregrado.pdf: 165 chunks

Ejemplo (primer chunk, 500 chars):
Departamento Administrativo de la Función Pública
Constitución Política 1 de 1991 Asamblea
Nacional Constituyente1 EVA - Gestor Normativo
Constitución Política 1 de 1991 Asamblea Nacional
Constituyente
Los datos publicados tienen propósitos exclusivamente informativos. El Departamento Administrativo de la Función Pública no se hace
responsable de la vigencia de la presente norma. Nos encontramos en un proceso permanente de actualización de los contenidos.
CONSTITUCIÓN POLITICA DE LA REPUBLICA DE


In [5]:
#%pip install -q --upgrade langchain-openai
!uv add langchain-openai --upgrade

⠇ jsonpointer==3.0.0                                                            ⠋ Resolving dependencies...                                                     ⠋ Resolving dependencies...                                                     Resolved 141 packages in 1.58s
⠙ Preparing packages... (0/7)                                                   Resolved 141 packages in 1.58s
⠙ Preparing packages... (0/7)                                                   
⠙ Preparing packages... (0/7)--------------     0 B/14.27 KiB           
⠙ Preparing packages... (0/7)---------- 14.27 KiB/14.27 KiB         
⠙ Preparing packages... (0/7)---------- 14.27 KiB/14.27 KiB         
typing-inspection    ------------------------------ 14.27 KiB/14.27 KiB
⠙ Preparing packages... (0/7)--------------     0 B/125.36 KiB          
typing-inspection    ------------------------------ 14.27 KiB/14.27 KiB
⠙ Preparing packages... (0/7)-------------- 16.00 KiB/125.36 KiB        
typing-inspection    ---------------

In [3]:
##SECCION 5 PARSES 
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# Definir los campos esperados en la salida
response_schemas = [
    ResponseSchema(name="answer", description="Respuesta concisa a la consulta legal en español."),
    ResponseSchema(name="citations", description="Lista de citas con formato [doc:filename, chunk:index]."),
    ResponseSchema(name="follow_up", description="true/false indicando si el usuario debería hacer una pregunta de seguimiento."),
]

# Crear el parser estructurado
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Instrucciones para el modelo
format_instructions = output_parser.get_format_instructions()

# Prompt template que exige formato
parser_prompt = PromptTemplate(
    template="""
Eres un asistente legal colombiano. Responde siempre en formato JSON.

Pregunta: {question}

{format_instructions}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

# Ejemplo de uso
question = "¿Qué artículo de la Constitución Colombiana protege la libertad de expresión?"

prompt_text = parser_prompt.format(question=question)
response = llm.invoke(prompt_text)

print("=== Respuesta cruda del LLM ===")
print(response.content)

try:
    parsed = output_parser.parse(response.content)
    print("\n=== Respuesta parseada (dict válido) ===")
    print(parsed)
except Exception as e:
    print("\n[ERROR] No se pudo parsear la salida:", e)


=== Respuesta cruda del LLM ===
```json
{
	"answer": "El artículo 20 de la Constitución Política de Colombia protege la libertad de expresión.",
	"citations": "[const:1991, chunk:20]",
	"follow_up": "false"
}
```

=== Respuesta parseada (dict válido) ===
{'answer': 'El artículo 20 de la Constitución Política de Colombia protege la libertad de expresión.', 'citations': '[const:1991, chunk:20]', 'follow_up': 'false'}


In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Definir un prompt que incluya historial de conversación
memory_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
Eres un asistente legal colombiano.
Responde en español de forma clara y breve.

Historial de conversación:
{history}

Nueva pregunta:
{question}

Respuesta:
"""
)

# Inicializar la memoria
memory = ConversationBufferMemory(memory_key="history")

# Construir el chain con memoria
conversation_chain = LLMChain(
    llm=llm,
    prompt=memory_prompt,
    memory=memory,
)

# Ejemplo de interacción multi-turno
print("Turno 1:")
resp1 = conversation_chain.run("Soy un estudiante de Ingenieria de sistemas y me llamo Wilson")
print(resp1)

print("\nTurno 2 (con memoria):")
resp2 = conversation_chain.run("¿Como me llamo y que estudio?")
print(resp2)



/var/folders/s9/z01_xhpd6gs53wyd4_pzp9z40000gn/T/ipykernel_11255/2131340476.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history")
/var/folders/s9/z01_xhpd6gs53wyd4_pzp9z40000gn/T/ipykernel_11255/2131340476.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation_chain = LLMChain(
/var/folders/s9/z01_xhpd6gs53wyd4_pzp9z40000gn/T/ipykernel_11255/2131340476.py:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp1 = conversation_chain.run("Soy un estudiante de Ingenieria de sistemas y me llamo Wilson")


Turno 1:
¡Hola, Wilson! Es un gusto.

¿En qué puedo ayudarte hoy?

Turno 2 (con memoria):
¡Hola, Wilson! Es un gusto.

¿En qué puedo ayudarte hoy?

Turno 2 (con memoria):
Te llamas Wilson y estudias Ingeniería de Sistemas.
Te llamas Wilson y estudias Ingeniería de Sistemas.


In [11]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_core.prompts import PromptTemplate

# Definición básica de rag_answer para evitar NameError
import textwrap

def rag_answer(query: str, k: int = 3):
    """Función básica de RAG usando DOCUMENT_CHUNKS."""
    if 'DOCUMENT_CHUNKS' not in globals() or not DOCUMENT_CHUNKS:
        return {'answer': 'No hay documentos cargados para buscar.', 'citations': []}
    
    q = query.lower()
    scored = []
    for c in DOCUMENT_CHUNKS:
        text = c.get('text', '') or ''
        score = text.lower().count(q)  # heurística simple
        scored.append({'text': text, 'score': score, 'metadata': {'source': c.get('source'), 'chunk_index': c.get('chunk_index')}})
    scored = sorted(scored, key=lambda x: x['score'], reverse=True)[:k]
    
    if not scored:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}
    
    context = '\n'.join([s['text'][:500] for s in scored])
    prompt = f"Eres un asistente legal colombiano. Responde en español basado en el contexto proporcionado.\n\nContexto:\n{context}\n\nPregunta: {query}\n\nRespuesta:"
    
    try:
        answer = llm.invoke(prompt).content
    except Exception as e:
        answer = f"Error al invocar LLM: {e}"
    
    return {'answer': answer, 'citations': []}

# --- Agente React Básico ---
def buscar_en_leyes(query: str) -> str:
    """Busca en la base de datos legal usando RAG."""
    res = rag_answer(query, k=3)
    return f"Respuesta (RAG): {res['answer']}"

tools = [
    Tool(
        name="BusquedaLegalRAG",
        func=buscar_en_leyes,
        description="Usa esta herramienta cuando la pregunta se relacione con leyes o documentos cargados."
    ),
    Tool(
        name="LLMGeneral",
        func=lambda q: llm.invoke(q).content,
        description="Usa esta herramienta cuando la pregunta NO sea legal o no requiera buscar documentos."
    )
]

# --- Inicializamos agente con tipo ReAct ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,   # muestra el razonamiento paso a paso
    max_iterations=3,
)

# --- Ejemplo de uso ---
print("Ejemplo 1: Pregunta legal (usa RAG)")
respuesta1 = agent.invoke("¿Qué artículo protege la libertad de expresión en Colombia?")
print(respuesta1)

print("\nEjemplo 2: Pregunta general (usa LLM)")
respuesta2 = agent.invoke("¿Quién fue Gabriel García Márquez?")
print(respuesta2)

Ejemplo 1: Pregunta legal (usa RAG)


> Entering new AgentExecutor chain...
La pregunta es de naturaleza legal, pero se enfoca en un derecho fundamental consagrado en la Constitución o en tratados internacionales, que suele ser información codificada y específica. Usaré la herramienta de búsqueda legal para confirmar el artículo exacto en la legislación colombiana.
Action: BusquedaLegalRAG
Action Input: artículo que protege la libertad de expresión en Colombia
Observation: Respuesta (RAG): No hay documentos cargados para buscar.
Thought:La pregunta es de naturaleza legal, pero se enfoca en un derecho fundamental consagrado en la Constitución o en tratados internacionales, que suele ser información codificada y específica. Usaré la herramienta de búsqueda legal para confirmar el artículo exacto en la legislación colombiana.
Action: BusquedaLegalRAG
Action Input: artículo que protege la libertad de expresión en Colombia
Observation: Respuesta (RAG): No hay documentos cargados para buscar

In [16]:
# ===== CELDA: Definir retrieve + rag_answer + re-crear agente ReAct =====
import warnings
warnings.filterwarnings('ignore')  # Suprimir warnings

import textwrap
import traceback

# 1) Comprobaciones básicas
if 'llm' not in globals():
    raise NameError("No existe la variable 'llm'. Inicializa tu LLM (ChatGroq/ChatOpenAI) antes de ejecutar esta celda.")

try:
    has_vectordb = vectordb is not None
except NameError:
    has_vectordb = False

has_chunks = ('DOCUMENT_CHUNKS' in globals() and bool(DOCUMENT_CHUNKS))

# 2) Funciones de recuperación (vectordb o fallback por keywords)
def retrieve_by_keywords(query: str, k: int = 4):
    """Fallback simple: cuenta ocurrencias en DOCUMENT_CHUNKS."""
    if not has_chunks:
        return []
    q = query.lower()
    words = q.split()  # dividir en palabras para mejor búsqueda
    scored = []
    for c in DOCUMENT_CHUNKS:
        text = c.get('text','') or ''
        score = sum(text.lower().count(word) for word in words)  # suma de ocurrencias de cada palabra
        scored.append({'text': text, 'score': score, 'metadata': {'source': c.get('source'), 'chunk_index': c.get('chunk_index')}})
    scored = sorted(scored, key=lambda x: x['score'], reverse=True)
    return scored[:k]

def retrieve_from_vectordb(query: str, k: int = 4):
    """Intentar usar vectordb (Chroma/FAISS) si está disponible."""
    try:
        # preferimos similarity_search_with_relevance_scores si existe
        if hasattr(vectordb, "similarity_search_with_relevance_scores"):
            results = vectordb.similarity_search_with_relevance_scores(query, k=k)
            out = []
            for doc, score in results:
                text = getattr(doc, "page_content", None) or (doc.get('text') if isinstance(doc, dict) else str(doc))
                meta = getattr(doc, "metadata", None) or (doc if isinstance(doc, dict) else {})
                out.append({'text': text, 'score': float(score), 'metadata': meta})
            return out
        else:
            results = vectordb.similarity_search(query, k=k)
            out = []
            for r in results:
                text = getattr(r, "page_content", None) or (r.get('text') if isinstance(r, dict) else str(r))
                meta = getattr(r, "metadata", None) or (r if isinstance(r, dict) else {})
                out.append({'text': text, 'score': None, 'metadata': meta})
            return out
    except Exception as e:
        print("⚠️ Error buscando en vectordb:", e)
        print("Se usará fallback por keywords (DOCUMENT_CHUNKS).")
        return retrieve_by_keywords(query, k=k)

def retrieve(query: str, k: int = 4):
    return retrieve_from_vectordb(query, k) if has_vectordb else retrieve_by_keywords(query, k)

# 3) Wrapper LLM compatible con .invoke() y llamada directa
def llm_call(prompt: str, **kwargs) -> str:
    """Llama al LLM y devuelve texto; maneja distintos formatos de respuesta."""
    try:
        resp = llm.invoke(prompt, **kwargs)
        return getattr(resp, "content", None) or str(resp)
    except Exception:
        try:
            resp = llm(prompt, **kwargs)
            return getattr(resp, "content", None) or str(resp)
        except Exception as e:
            return f"[ERROR en LLM]: {e}"

# 4) rag_answer robusto: recupera, arma contexto y llama al LLM
def rag_answer(query: str, k: int = 4):
    docs = retrieve(query, k)
    if not docs:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}

    ctx_parts = []
    citations = []
    for d in docs:
        text = d.get('text') if isinstance(d, dict) else str(d)
        meta = d.get('metadata', {}) if isinstance(d, dict) else {}
        src = meta.get('source', meta.get('filename', 'unknown'))
        idx = meta.get('chunk_index')
        score = d.get('score')
        citations.append({'source': src, 'chunk_index': idx, 'score': score})
        snippet = textwrap.shorten(text, width=1000, placeholder='...')
        ctx_parts.append(f"Source: {src} (chunk {idx}, score {score})\n{snippet}")

    context = "\n\n".join(ctx_parts)
    final_prompt = (
        "Eres un asistente legal. Usa SOLO el contexto provisto para responder la pregunta. "
        "Responde en español, de forma concisa, y al final incluye las citas en formato [source:filename chunk_index].\n\n"
        f"Contexto:\n{context}\n\n"
        f"Pregunta: {query}\n\n"
        "Respuesta:"
    )

    answer_text = llm_call(final_prompt)
    return {'answer': answer_text, 'citations': citations}

# 5) Reconstruir agente ReAct usando la herramienta que llama a rag_answer
from langchain.agents import initialize_agent, Tool, AgentType

def buscar_en_leyes_tool(q: str) -> str:
    try:
        r = rag_answer(q, k=3)
        return r.get('answer', '')
    except Exception as e:
        return f"[ERROR herramienta RAG]: {e}"

def llm_general_tool(q: str) -> str:
    try:
        return llm_call(q)
    except Exception as e:
        return f"[ERROR herramienta LLMGeneral]: {e}"

tools = [
    Tool(name="BusquedaLegalRAG", func=buscar_en_leyes_tool,
         description="Buscar en documentos legales indexados. Útil para consultas concretas sobre leyes y artículos."),
    Tool(name="LLMGeneral", func=llm_general_tool,
         description="Responder con el LLM sin usar la base de datos (general knowledge).")
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5  # Aumentado para evitar límite de iteraciones
)

print("✅ Definido rag_answer y recreado 'agent' con la herramienta RAG. Prueba con agent.run(...)")

✅ Definido rag_answer y recreado 'agent' con la herramienta RAG. Prueba con agent.run(...)


In [17]:
# Pruebas rápidas
print(agent.run("¿Qué artículo protege la libertad de expresión en Colombia?"))
#print(agent.run("Resume brevemente el concepto de habeas corpus."))




> Entering new AgentExecutor chain...
Thought: Necesito identificar el artículo de la legislación colombiana que protege la libertad de expresión. La herramienta *BusquedaLegalRAG* es la más adecuada para esta consulta específica sobre leyes y artículos.

Action: BusquedaLegalRAG
Action Input: artículo que protege la libertad de expresión en Colombia
Observation: No se encontraron documentos relevantes.
Thought:Thought: Necesito identificar el artículo de la legislación colombiana que protege la libertad de expresión. La herramienta *BusquedaLegalRAG* es la más adecuada para esta consulta específica sobre leyes y artículos.

Action: BusquedaLegalRAG
Action Input: artículo que protege la libertad de expresión en Colombia
Observation: No se encontraron documentos relevantes.
Thought:Thought: La herramienta *BusquedaLegalRAG* no arrojó resultados, lo cual puede indicar que la base de datos es limitada. Consultaré con el modelo de conocimiento general (*LLMGeneral*) para obtener el artíc